In [11]:
import calendar
import pandas as pd

In [16]:
import plotly.express as px

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import helper

### Driver Code

In [15]:
greece_df = helper.make_connection('mobility_stats_apple', 'region', 'Greece')
print('-------------')
greece_df = helper.clean_df(greece_df)

Connecting to the PostgreSQL database...
Connection successful
Version: ('PostgreSQL 14.0, compiled by Visual C++ build 1914, 64-bit',)
Fetching Data from database...
Data Fetched
-------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   region   694 non-null    object        
 1   date     694 non-null    datetime64[ns]
 2   driving  694 non-null    float64       
 3   transit  0 non-null      object        
 4   walking  694 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 27.2+ KB

Checking for null columns...
-- Dropped null column: transit

Clearing null rows...


In [7]:
greece_df['day_name'] = greece_df['date'].dt.day_name()
day_by_day_trends = greece_df.groupby(by='day_name').mean().reset_index()

day_by_day_trends.loc[:, 'day_name'] = pd.Categorical(day_by_day_trends['day_name'], categories=list(calendar.day_name), ordered=True)
day_by_day_trends.sort_values('day_name', inplace=True)

In [34]:
fig = px.density_heatmap(
    data_frame=greece_df,
    x='driving',
    y='walking',
    marginal_x="violin",
    marginal_y="violin",
    width=800,
    height=800
)

fig.show()
